Image 전체 면적 대비, 손상부위가 차지하는 비율로 계산하여 손상정도 정의 

In [1]:
import os
import cv2
import numpy as np
import random
from typing import List, Dict, Tuple, Optional

def _ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def _read_image(path: str, flags: int = cv2.IMREAD_UNCHANGED) -> np.ndarray:
    img = cv2.imread(path, flags)
    if img is None:
        raise FileNotFoundError(f"Failed to read image: {path}")
    return img

def _rotate_affine(img: np.ndarray, angle: float) -> np.ndarray:
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    return cv2.warpAffine(
        img, M, (w, h),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=0
    )

def damage_creation(
    image_list: List[str],
    output_dir: str,
    damage_image_path: str,
    target_damage_ratios: Optional[Dict[str, float]] = None,
    scale_min: float = 0.1,
    scale_max: float = 0.6,
    max_iters_per_level: int = 2000,
    jpeg_quality: int = 95,
    random_seed: Optional[int] = None,
    area_threshold_mode: bool = False,  # True면 면적(>0.5) 기준, False면 알파적분 기준
) -> None:
    """
    원본 이미지들에 손상 패턴을 랜덤 합성해 Minor/Moderate/Severe 버전을 생성합니다.

    Args:
        image_list: 원본 이미지 경로 리스트.
        output_dir: 출력 루트 디렉터리.
        damage_image_path: 손상 소스 이미지(알파 포함 4채널 PNG 권장).
        target_damage_ratios: 각 레벨의 목표 손상 비율(dict). None이면 기본값 사용.
        scale_min, scale_max: 손상 패치 랜덤 스케일 범위.
        max_iters_per_level: 레벨당 합성 시도 최대 횟수(무한루프 방지).
        jpeg_quality: 저장 JPEG 품질(1~100).
        random_seed: 랜덤 시드(재현성).
        area_threshold_mode: True면 (mask>0.5) 비율, False면 알파 적분/총픽셀.
    """
    if random_seed is not None:
        random.seed(random_seed)
        np.random.seed(random_seed)

    # 출력 폴더
    for level in ("Minor", "Moderate", "Severe"):
        _ensure_dir(os.path.join(output_dir, level))

    # 손상 이미지 로드 및 검증
    damage_image = _read_image(damage_image_path, cv2.IMREAD_UNCHANGED)
    if damage_image.ndim != 3 or damage_image.shape[2] < 4:
        raise ValueError("damage_image must have an alpha channel (4-channel image).")

    damage_rgb = damage_image[:, :, :3]
    damage_alpha = damage_image[:, :, 3].astype(np.float32) / 255.0

    # 기본 목표 비율
    if target_damage_ratios is None:
        target_damage_ratios = {"Minor": 0.05, "Moderate": 0.10, "Severe": 0.20}

    for original_image_path in image_list:
        original_name = os.path.splitext(os.path.basename(original_image_path))[0]
        original = _read_image(original_image_path, cv2.IMREAD_COLOR)
        H, W = original.shape[:2]

        # 손상 패치 최대 스케일 상한은 원본 대비 크기 보정
        scale_upper = min(scale_max, W / damage_rgb.shape[1], H / damage_rgb.shape[0])
        if scale_upper < scale_min:
            # 원본이 너무 작거나 손상 소스가 지나치게 큰 경우
            scale_upper = max(scale_min, scale_upper)
        scale_upper = max(scale_upper, scale_min)

        for level, target_ratio in target_damage_ratios.items():
            damaged = original.copy()
            mask_acc = np.zeros((H, W), dtype=np.float32)  # 0~1 누적
            current_ratio = 0.0

            iters = 0
            while current_ratio < target_ratio and iters < max_iters_per_level:
                iters += 1

                scale = random.uniform(scale_min, scale_upper)
                angle = random.uniform(0, 360)

                # 리사이즈
                resized_rgb = cv2.resize(damage_rgb, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
                resized_alpha = cv2.resize(damage_alpha, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

                # 회전
                rot_rgb = _rotate_affine(resized_rgb, angle)
                rot_alpha = _rotate_affine(resized_alpha, angle)

                h, w = rot_rgb.shape[:2]
                if h == 0 or w == 0:
                    continue

                # 랜덤 위치
                x0 = random.randint(0, max(1, W - w))
                y0 = random.randint(0, max(1, H - h))

                # 경계 맞춤(방어)
                hh = min(h, H - y0)
                ww = min(w, W - x0)
                if hh <= 0 or ww <= 0:
                    continue

                patch_rgb = rot_rgb[:hh, :ww]
                patch_a = rot_alpha[:hh, :ww]

                # 누적 마스크 업데이트(클리핑)
                mask_acc[y0:y0+hh, x0:x0+ww] = np.clip(mask_acc[y0:y0+hh, x0:x0+ww] + patch_a, 0.0, 1.0)

                # 손상 비율 계산
                if area_threshold_mode:
                    current_ratio = float((mask_acc > 0.5).sum()) / float(W * H)
                else:
                    current_ratio = float(mask_acc.sum()) / float(W * H)

                # 알파 3채널화 및 블렌딩(브로드캐스팅)
                a3 = patch_a[..., None]  # (hh, ww, 1)
                roi = damaged[y0:y0+hh, x0:x0+ww, :].astype(np.float32)
                damaged[y0:y0+hh, x0:x0+ww, :] = (a3 * patch_rgb.astype(np.float32) + (1.0 - a3) * roi)

            # 최종 저장
            save_path = os.path.join(output_dir, level, f"{original_name}_{level}.jpg")
            ok = cv2.imwrite(save_path, damaged.astype(np.uint8), [int(cv2.IMWRITE_JPEG_QUALITY), int(jpeg_quality)])
            if not ok:
                raise IOError(f"Failed to save image: {save_path}")

            print(f"{original_name} [{level}] - ratio≈{current_ratio:.3f}, iters={iters}")

    print("각 손상 정도 이미지 생성 완료")

In [ ]:
import glob

# 원본 이미지 경로 리스트 (예: sample.jpg 1장)
image_list = ["./sample/Add_01.jpg"]

# 손상 이미지 (알파채널 포함 PNG, 예: crack_overlay.png)
damage_image_path = "./damage/damage.png"

# 출력 폴더 지정
output_dir = "./output_damaged"

# 함수 호출
damage_creation(
    image_list=image_list,
    output_dir=output_dir,
    damage_image_path=damage_image_path,
    random_seed=42
)

Add_01 [Minor] - ratio≈0.074, iters=2
Add_01 [Moderate] - ratio≈0.109, iters=3
Add_01 [Severe] - ratio≈0.232, iters=8
각 손상 정도 이미지 생성 완료


실제 단어가 있는 Bounding Box를 손상부위가 차지하는 비율로 계산여 손상정도 정의 하는경우

In [ ]:
import os
import json
import cv2
import numpy as np
import random
from typing import List, Dict, Tuple, Optional


BBox = Tuple[int, int, int, int]  # (x1, y1, x2, y2)

def _ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def _read_image(path: str, flags: int = cv2.IMREAD_UNCHANGED) -> np.ndarray:
    img = cv2.imread(path, flags)
    if img is None:
        raise FileNotFoundError(f"Failed to read image: {path}")
    return img

def _rotate_affine(img: np.ndarray, angle: float) -> np.ndarray:
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    return cv2.warpAffine(
        img, M, (w, h),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=0
    )

def _draw_target_mask(H: int, W: int, bboxes: List[BBox], margin_px: int = 0) -> np.ndarray:
    mask = np.zeros((H, W), dtype=np.float32)
    for (x1, y1, x2, y2) in bboxes:
        x1 = max(0, x1 - margin_px); y1 = max(0, y1 - margin_px)
        x2 = min(W, x2 + margin_px); y2 = min(H, y2 + margin_px)
        if x2 > x1 and y2 > y1:
            mask[y1:y2, x1:x2] = 1.0
    return mask


def parse_label_file(
    label_txt_path: str,
    images_base_dir: Optional[str] = None,
    use_only_non_difficult: bool = True,
    include_prefixes: Optional[List[str]] = None,  # 예: ["K_", "V_", "G_"]
    exclude_prefixes: Optional[List[str]] = None,
) -> Tuple[List[str], Dict[str, List[BBox]]]:
    """
    label.txt 포맷 (한 줄):
      <image_path>\t[{"transcription": "...", "points": [[x,y],... x4], "difficult": false}, ...]
    - points는 4점 폴리곤이므로 축정렬 bbox로 변환: min/max 이용
    - include/exclude_prefixes로 transcription 필터링 가능
    """
    image_list: List[str] = []
    bboxes_per_image: Dict[str, List[BBox]] = {}

    with open(label_txt_path, "r", encoding="utf-8") as f:
        for ln, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue

            # 이미지 경로 \t JSON 문자열
            try:
                img_path_raw, json_str = line.split("\t", 1)
            except ValueError:
                print(f"[WARN] Line {ln}: cannot split by tab. Skipped.")
                continue

            # 이미지 경로 정규화
            img_path = img_path_raw
            if images_base_dir and not os.path.isabs(img_path):
                img_path = os.path.join(images_base_dir, img_path_raw)

            # 어레이 파싱
            try:
                items = json.loads(json_str)
            except json.JSONDecodeError as e:
                print(f"[WARN] Line {ln}: JSON decode error ({e}). Skipped.")
                continue

            # 각 항목에서 bbox 추출
            bboxes: List[BBox] = []
            for it in items:
                # difficult 필터
                if use_only_non_difficult and bool(it.get("difficult", False)):
                    continue

                trans = str(it.get("transcription", ""))

                # prefix 필터: include → 허용 목록만, exclude → 제외
                if include_prefixes and not any(trans.startswith(p) for p in include_prefixes):
                    continue
                if exclude_prefixes and any(trans.startswith(p) for p in exclude_prefixes):
                    continue

                pts = it.get("points", None)
                if not pts or len(pts) < 4:
                    continue

                # 축정렬 bbox로 변환
                xs = [int(round(p[0])) for p in pts]
                ys = [int(round(p[1])) for p in pts]
                x1, y1 = max(0, min(xs)), max(0, min(ys))
                x2, y2 = max(xs), max(ys)
                if x2 > x1 and y2 > y1:
                    bboxes.append((x1, y1, x2, y2))

            if not bboxes:
                # 박스가 하나도 없으면 스킵
                print(f"[INFO] {img_path_raw}: no valid boxes after filtering.")
                continue

            image_list.append(img_path)
            bboxes_per_image[img_path] = bboxes

    return image_list, bboxes_per_image

def damage_creation_bbox_based(
    image_list: List[str],
    output_dir: str,
    damage_image_path: str,
    bboxes_per_image: Dict[str, List[BBox]],
    target_damage_ratios: Optional[Dict[str, float]] = None,
    scale_min: float = 0.1,
    scale_max: float = 0.6,
    max_iters_per_level: int = 2000,
    jpeg_quality: int = 95,
    random_seed: Optional[int] = None,
    restrict_to_bboxes: bool = True,
    bbox_margin_px: int = 2,
) -> None:
    """
    글자 Bounding Box 기준 손상 비율 = (손상∩글자영역) / (글자영역)
    """
    if random_seed is not None:
        random.seed(random_seed)
        np.random.seed(random_seed)

    for level in ("Minor", "Moderate", "Severe"):
        _ensure_dir(os.path.join(output_dir, level))

    damage_image = _read_image(damage_image_path, cv2.IMREAD_UNCHANGED)
    if damage_image.ndim != 3 or damage_image.shape[2] < 4:
        raise ValueError("damage_image must have an alpha channel (4-channel image).")

    damage_rgb = damage_image[:, :, :3]
    damage_alpha = damage_image[:, :, 3].astype(np.float32) / 255.0

    if target_damage_ratios is None:
        target_damage_ratios = {"Minor": 0.05, "Moderate": 0.10, "Severe": 0.20}

    for original_image_path in image_list:
        original_name = os.path.splitext(os.path.basename(original_image_path))[0]
        original = _read_image(original_image_path, cv2.IMREAD_COLOR).astype(np.float32)
        H, W = original.shape[:2]

        # 이 이미지의 타깃 마스크
        bboxes = bboxes_per_image.get(original_image_path, [])
        if not bboxes:
            print(f"[WARN] No bboxes for {original_image_path}. Skipping.")
            continue

        target_mask = _draw_target_mask(H, W, bboxes, margin_px=bbox_margin_px)
        target_area = float(target_mask.sum())
        if target_area <= 1.0:
            print(f"[WARN] Very small target area for {original_image_path}. Skipping.")
            continue

        # 손상 패치 스케일 상한
        scale_upper = min(scale_max, W / damage_rgb.shape[1], H / damage_rgb.shape[0])
        scale_upper = max(scale_upper, scale_min)

        for level, target_ratio in target_damage_ratios.items():
            damaged = original.copy()
            mask_acc = np.zeros((H, W), dtype=np.float32)
            current_ratio = 0.0
            iters = 0

            while current_ratio < target_ratio and iters < max_iters_per_level:
                iters += 1
                scale = random.uniform(scale_min, scale_upper)
                angle = random.uniform(0, 360)

                resized_rgb = cv2.resize(damage_rgb, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
                resized_alpha = cv2.resize(damage_alpha, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

                rot_rgb = _rotate_affine(resized_rgb, angle).astype(np.float32)
                rot_alpha = _rotate_affine(resized_alpha, angle).astype(np.float32)

                h, w = rot_rgb.shape[:2]
                if h == 0 or w == 0:
                    continue

                # 배치 위치 (간단 랜덤; 필요하면 target_mask의 nonzero 근처로 샘플링하도록 개선 가능)
                x0 = random.randint(0, max(1, W - w))
                y0 = random.randint(0, max(1, H - h))
                hh = min(h, H - y0)
                ww = min(w, W - x0)
                if hh <= 0 or ww <= 0:
                    continue

                patch_rgb = rot_rgb[:hh, :ww]
                patch_a = rot_alpha[:hh, :ww]

                if restrict_to_bboxes:
                    patch_target = target_mask[y0:y0+hh, x0:x0+ww]
                    patch_a = patch_a * patch_target  # 글자 영역 밖 0

                # 누적 마스크 업데이트
                prev = mask_acc[y0:y0+hh, x0:x0+ww]
                mask_acc[y0:y0+hh, x0:x0+ww] = np.clip(prev + patch_a, 0.0, 1.0)

                # 글자영역 교집합 비율
                intersect = np.minimum(mask_acc, target_mask).sum()
                current_ratio = float(intersect) / float(target_area)

                # 알파 블렌딩
                a3 = patch_a[..., None]
                roi = damaged[y0:y0+hh, x0:x0+ww, :]
                damaged[y0:y0+hh, x0:x0+ww, :] = a3 * patch_rgb + (1.0 - a3) * roi

            save_path = os.path.join(output_dir, level, f"{original_name}_{level}.jpg")
            ok = cv2.imwrite(save_path, damaged.astype(np.uint8),
                              [int(cv2.IMWRITE_JPEG_QUALITY), int(jpeg_quality)])
            if not ok:
                raise IOError(f"Failed to save image: {save_path}")

            print(f"{original_name} [{level}] - text-damage ratio≈{current_ratio:.3f}, iters={iters}")

    print("글자영역 기준 손상 이미지 생성 완료")

In [6]:
label_txt = "./label.txt"                 # 질문에서 주신 포맷의 파일
images_base_dir = ""                     # 이미지가 label 경로 기준 상대경로라면 베이스 디렉토리 지정
include_prefixes = None                   # 예: ["K_", "V_", "G_"] ; None이면 모두 포함
exclude_prefixes = None                   # 예: ["G_"] 등 제외할 접두사

image_list, bboxes_per_image = parse_label_file(
    label_txt_path=label_txt,
    images_base_dir=images_base_dir,
    use_only_non_difficult=True,
    include_prefixes=include_prefixes,
    exclude_prefixes=exclude_prefixes,
)

In [12]:
output_dir = "./out_nameplate"
damage_image_path = "./damage/damage.png"  # 4채널 PNG(알파 포함) 필요
target_damage = {"Minor": 0.10, "Moderate": 0.20, "Severe": 0.30}

image_list = image_list[:1]  # 테스트용으로 1장만

damage_creation_bbox_based(
    image_list=image_list,
    output_dir=output_dir,
    damage_image_path=damage_image_path,
    bboxes_per_image=bboxes_per_image,
    target_damage_ratios=target_damage,
    scale_min=0.12,
    scale_max=0.7,
    max_iters_per_level=25,
    jpeg_quality=95,
    random_seed=0,
    restrict_to_bboxes=False,  # True => 글자영역 내부에만 손상 False => 글자영역 외부도 손상 가능
    bbox_margin_px=2,
)

Add_01 [Minor] - text-damage ratio≈0.112, iters=2
Add_01 [Moderate] - text-damage ratio≈0.208, iters=5
Add_01 [Severe] - text-damage ratio≈0.305, iters=7
✅ 글자영역 기준 손상 이미지 생성 완료
